In [148]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [149]:
train = pd.read_csv(r"C:\Users\Admin\Desktop\nlpdisaster\train.csv")
test = pd.read_csv(r"C:\Users\Admin\Desktop\nlpdisaster\test.csv")
samp = pd.read_csv(r"C:\Users\Admin\Desktop\nlpdisaster\sample_submission.csv")

In [150]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [151]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [152]:
samp.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [153]:
print('Train shape:', train.shape)
print('Test shape:', test.shape)

Train shape: (7613, 5)
Test shape: (3263, 4)


In [154]:
train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [155]:
test.isnull().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [156]:
train['location'].unique()

array([nan, 'Birmingham', 'Est. September 2012 - Bristol', ...,
       'Vancouver, Canada', 'London ', 'Lincoln'], dtype=object)

In [157]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [158]:
train.drop(['location', 'keyword', 'id'], axis=1, inplace=True)
test.drop(['location', 'keyword', 'id'], axis=1, inplace=True)

### Splitting Data

In [159]:
train_size = round(train.shape[0]*0.8)
test_size = train.shape[0] - train_size
print(train_size)
print(test_size)


X = train.drop(['target'],axis=1)
y = train['target']

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

6090
1523
(6090, 1) (1523, 1)
(6090,) (1523,)


In [160]:
y1 = train['target']
y_train1, y_test1 = y1[:train_size], y1[train_size:]
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
y_test1 = LE.fit_transform(y_test1)

### Text Preprocessing

In [161]:
import re
X_train['text'] = X_train['text'].apply(lambda x: re.split('http:\/\/.*', str(x))[0])
X_test['text'] = X_test['text'].apply(lambda x: re.split('http:\/\/.*', str(x))[0])

test['text'] = test['text'].apply(lambda x: re.split('http:\/\/.*', str(x))[0])

In [162]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [163]:
X_train['text'] = X_train['text'].apply(lambda x: decontracted(x))
X_test['text'] = X_test['text'].apply(lambda x: decontracted(x))

test['text'] = test['text'].apply(lambda x: decontracted(x))

In [164]:
#Removing Punctuations

X_train.replace('[^a-zA-Z]',' ', regex=True, inplace=True)
X_test.replace('[^a-zA-Z]',' ', regex=True, inplace=True)

test.replace('[^a-zA-Z]',' ', regex=True, inplace=True)

In [165]:
#Converting to lower case characters

for index in X_train.columns:
  X_train[index] = X_train[index].str.lower()

for index in X_test.columns:
  X_test[index] = X_test[index].str.lower()

for index in test.columns:
  test[index] = test[index].str.lower()

In [166]:
X_test.head()

,text
6090,nowplaying sinking fast now or never on nor...
6091,that horrible sinking feeling when you ve be...
6092,nigga car sinking but he snapping it up for fo...
6093,abandonedpics you should delete this one it i...
6094,that horrible sinking feeling when you ve be...


In [167]:
#Removing multiple blank spaces

X_train = X_train.replace('\s+', ' ', regex=True)
X_test = X_test.replace('\s+', ' ', regex=True)

test = test.replace('\s+', ' ', regex=True)

In [168]:
#Removing one letter words

X_train['text'] = X_train['text'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
X_test['text'] = X_test['text'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')

test['text'] = test['text'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')

In [169]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [170]:
stop_words = set(stopwords.words('english')) 
len(stop_words)
X_train['text'] = X_train['text'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))
X_test['text'] = X_test['text'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

test['text'] = test['text'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

In [171]:
stemmer = nltk.SnowballStemmer(language='english')

X_train['text'] = X_train['text'].apply(lambda x: ' '.join(stemmer.stem(term) for term in x.split()))
X_test['text'] = X_test['text'].apply(lambda x: ' '.join(stemmer.stem(term) for term in x.split()))

test['text'] = test['text'].apply(lambda x: ' '.join(stemmer.stem(term) for term in x.split()))

In [172]:
import nltk
nltk.download('wordnet')

from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
lmtzr.lemmatize('ladies')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'lady'

In [173]:
X_train['text'] = X_train['text'].apply(lambda x: ' '.join(lmtzr.lemmatize(term) for term in x.split()))
X_test['text'] = X_test['text'].apply(lambda x: ' '.join(lmtzr.lemmatize(term) for term in x.split()))

test['text'] = test['text'].apply(lambda x: ' '.join(lmtzr.lemmatize(term) for term in x.split()))

In [174]:
X_train.head()

,text
0,deed reason earthquak may allah forgiv u
1,forest fire near la rong sask canada
2,resid ask ishelt place notifi offic evacu shel...
3,peopl receiv wildfir evacu order california
4,got sent photo rubi alaska smoke wildfir pour ...


In [175]:
train_lines = []
for row in range(0,X_train.shape[0]):
  train_lines.append(' '.join(str(x) for x in X_train.iloc[row,:]))

test_lines = []
for row in range(0,X_test.shape[0]):
  test_lines.append(' '.join(str(x) for x in X_test.iloc[row,:]))

predtest_lines = []
for row in range(0,test.shape[0]):
  predtest_lines.append(' '.join(str(x) for x in test.iloc[row,:]))

In [176]:
train_lines

['deed reason earthquak may allah forgiv u',
 'forest fire near la rong sask canada',
 'resid ask ishelt place notifi offic evacu shelter place order expect',
 'peopl receiv wildfir evacu order california',
 'got sent photo rubi alaska smoke wildfir pour school',
 'rockyfir updat california hwi close direct due lake counti fire cafir wildfir',
 'flood disast heavi rain caus flash flood street manitou colorado spring area',
 'top hill see fire wood',
 'emerg evacu happen build across street',
 'afraid tornado come area',
 'three peopl die heat wave far',
 'haha south tampa get flood hah wait second live south tampa gonna gonna fvck flood',
 'rain flood florida tampabay tampa day lost count',
 'flood bago myanmar arriv bago',
 'damag school bus multi car crash break',
 'man',
 'love fruit',
 'summer love',
 'car fast',
 'goooooooaaaaaal',
 'ridicul',
 'london cool',
 'love ski',
 'wonder day',
 'looooool',
 'way eat shit',
 'nyc last week',
 'love girlfriend',
 'cooool',
 'like pasta',
 

In [177]:
test_lines

['nowplay sink fast never north east unsign radio listen',
 'horribl sink feel home phone realis whole time',
 'nigga car sink snap fox prioriti',
 'abandonedp delet one abbandon sink darsena castello scaligero di sirmion',
 'horribl sink feel home phone realis whole time',
 'equip ego break upon dig famili internet hoke except versus sink term dfljev',
 'currenc transgress payday prison ward sink fund payment unsecur loan jbumzqpk',
 'horribl sink feel home phone realis whole time',
 'lost alon sink like stone carri onnnn',
 'lie clinton sink donald trump sing let make america great http co zv chjclf',
 'ap slow report sink boat mediterranean sea shame',
 'walk plank sink ship',
 'sink ship sinkingshipindi scarlet lane lenor replac stone saison stonebrewingco',
 'horribl sink feel home phone realis whole time',
 'movi titan jack rose could stay wooden beam without sink',
 'lost amp alon sink like stone carri',
 'chanc get gander sink ship tna ca help appeas morbid curio destinationimp

In [178]:
from sklearn.feature_extraction.text import CountVectorizer

countvector = CountVectorizer()
X_train_cv = countvector.fit_transform(train_lines)
X_test_cv = countvector.transform(test_lines)

test_cv = countvector.transform(predtest_lines)

In [179]:
#Using TfidfVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

tfidfvector = TfidfTransformer()
X_train_tf = tfidfvector.fit_transform(X_train_cv)
X_test_tf = tfidfvector.fit_transform(X_test_cv)

test_tf = tfidfvector.fit_transform(test_cv)

### Naive-Bayes

In [180]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(X_train_tf, y_train)

In [181]:
X_train_tf.shape, X_test_tf.shape

((6090, 11308), (1523, 11308))

In [182]:
preds = clf.predict(X_test_tf)
preds

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [183]:
y_test.reset_index(drop=True, inplace=True)

In [184]:
n_right = 0
for i in range(len(preds)):
    if preds[i] == y_test[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(y_test)) * 100)))

Accuracy: 78.59%


### Logistic-Regression one vs all Classification

In [186]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier # Binary Relevance

lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

In [187]:
clf.fit(X_train_tf, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [188]:
preds = clf.predict(X_test_tf)
preds

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [189]:
n_right = 0
for i in range(len(preds)):
    if preds[i] == y_test[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(y_test)) * 100)))

Accuracy: 78.99%


In [102]:
predss = clf.predict(test_tf)

### Support Vector Machine

In [190]:
from sklearn.svm import SVC

clf = SVC(kernel='linear').fit(X_train_tf, y_train)

In [191]:
preds = clf.predict(X_test_tf)
preds

array([0, 0, 1, ..., 1, 1, 1], dtype=int64)

In [192]:
n_right = 0
for i in range(len(preds)):
    if preds[i] == y_test[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(y_test)) * 100)))

Accuracy: 78.40%


In [ ]:
len(preds)

1523

In [ ]:
test1 = pd.read_csv('/content/drive/My Drive/Kaggle projects/Real or Not? NLP with Disaster Tweets/test.csv')

In [ ]:
submit = pd.DataFrame({'id': test1.id, 'target': predss})

In [ ]:
submit.to_csv('submission888.csv', index=False)